# Hyperparameter Tuning for NLP Models

This notebook focuses on the hyperparameter tuning of five different transformer-based models to optimize their performance for question answering task.

## Models Used
- **RoBERTa**: A robustly optimized BERT approach that modifies key hyperparameters in BERT and removes the next-sentence pretraining objective.
- **DistilBERT**: A distilled version of BERT that retains most of the performance of BERT but with fewer parameters and faster inference time.
- **XLM-RoBERTa**: A multilingual model trained on 100 different languages, suitable for cross-lingual transfer learning.

## Objectives
- Identify the best hyperparameters for each model based on F1-score and exact matching.
- Re-tune the models for multiclass classification on bias type.
- Compare the performance across models and classification tasks to determine the most effective model configuration.

Each section of the notebook will guide you through the process of data preparation, model configuration, hyperparameter tuning, and evaluation of results.


## 0. Imports, libraries and rusable functions

In [4]:
# Standard Library Imports
import ast
import copy
import csv
import json
import math
import os
import re
import time
import warnings
import logging
import random
import collections
from collections import Counter
from typing import List, Tuple, Optional
from IPython.display import HTML, display
import math
import time
from unidecode import unidecode


# Data Handling Libraries
import numpy as np
import pandas as pd
import csv
from torch.utils.data import random_split
import datasets
from datasets import ClassLabel, Sequence

# Data Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
# import scikitplot as skplt  # Uncomment if scikit-plot is installed and needed

# Machine Learning: Model Preparation
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import cross_val_score, cross_validate, KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler

# Machine Learning: Models and Frameworks
import tensorflow as tf
import torch
import evaluate
import xgboost
import wandb
from xgboost import plot_importance  # Uncomment if xgboost importance plot is required


# NLP and Transformers
from transformers import (AdamW, AutoModelForSequenceClassification, AutoModelForQuestionAnswering,
                          AutoTokenizer, CamembertForSequenceClassification, DistilBertConfig,
                          DistilBertForSequenceClassification, DistilBertModel, EarlyStoppingCallback,
                          get_linear_schedule_with_warmup, RobertaForSequenceClassification, EvalPrediction,
                          Trainer, TrainerCallback, TrainingArguments, XLMRobertaForSequenceClassification,
                         DefaultDataCollator, BertForQuestionAnswering, DataCollatorWithPadding, PreTrainedTokenizerFast,
                         default_data_collator, is_torch_xla_available)
from datasets import Dataset, DatasetDict, load_dataset
from transformers.trainer_utils import PredictionOutput, speed_metrics

# Experiment Tracking and Optimization Utilities
import optuna
from optuna.trial import TrialState
# import wandb  # Uncomment if using Weights & Biases for experiment tracking

# Progress Bar Utilities
from tqdm.notebook import tqdm


In [5]:
#wandb.login(key='8f7092f0fdaf14add2b4cc07cb0e740080cdd8e7')
wandb.login()

wandb: Currently logged in as: mzak071 (COMPSCI714). Use `wandb login --relogin` to force relogin


True

In [6]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")

### Compute_metrics function for Question and Answering problem is different to classification, more preocessing required.

metric = evaluate.load("squad_v2")

def compute_metrics(p: EvalPrediction):
        return metric.compute(predictions=p.predictions, references=p.label_ids)

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

GPU: NVIDIA GeForce RTX 4070 Ti SUPER is available.
cuda


In [7]:
"""
A subclass of `Trainer` specific to Question-Answering tasks
"""

if is_torch_xla_available():
    import torch_xla.core.xla_model as xm
    import torch_xla.debug.metrics as met


class QuestionAnsweringTrainer(Trainer):
    def __init__(self, *args, eval_examples=None, post_process_function=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_examples = eval_examples
        self.post_process_function = post_process_function

    def evaluate(self, eval_dataset=None, eval_examples=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        eval_dataset = self.eval_dataset if eval_dataset is None else eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        eval_examples = self.eval_examples if eval_examples is None else eval_examples

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                eval_dataloader,
                description="Evaluation",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )
        if self.post_process_function is not None and self.compute_metrics is not None and self.args.should_save:
            # Only the main node write the results by default
            eval_preds = self.post_process_function(eval_examples, eval_dataset, output.predictions)
            metrics = self.compute_metrics(eval_preds)

            # Prefix all keys with metric_key_prefix + '_'
            for key in list(metrics.keys()):
                if not key.startswith(f"{metric_key_prefix}_"):
                    metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
            metrics.update(output.metrics)
        else:
            metrics = output.metrics

        if self.args.should_log:
            # Only the main node log the results by default
            self.log(metrics)

        if self.args.tpu_metrics_debug or self.args.debug:
            # tpu-comment: Logging debug metrics for PyTorch/XLA (compile, execute times, ops, etc.)
            xm.master_print(met.metrics_report())

        self.control = self.callback_handler.on_evaluate(self.args, self.state, self.control, metrics)
        return metrics

    def predict(self, predict_dataset, predict_examples, ignore_keys=None, metric_key_prefix: str = "test"):
        predict_dataloader = self.get_test_dataloader(predict_dataset)

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else self.evaluation_loop
        start_time = time.time()
        try:
            output = eval_loop(
                predict_dataloader,
                description="Prediction",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
                metric_key_prefix=metric_key_prefix,
            )
        finally:
            self.compute_metrics = compute_metrics
        total_batch_size = self.args.eval_batch_size * self.args.world_size
        if f"{metric_key_prefix}_jit_compilation_time" in output.metrics:
            start_time += output.metrics[f"{metric_key_prefix}_jit_compilation_time"]
        output.metrics.update(
            speed_metrics(
                metric_key_prefix,
                start_time,
                num_samples=output.num_samples,
                num_steps=math.ceil(output.num_samples / total_batch_size),
            )
        )

        if self.post_process_function is None or self.compute_metrics is None:
            return output

        predictions = self.post_process_function(predict_examples, predict_dataset, output.predictions, "predict")
        metrics = self.compute_metrics(predictions)

        # Prefix all keys with metric_key_prefix + '_'
        for key in list(metrics.keys()):
            if not key.startswith(f"{metric_key_prefix}_"):
                metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)
        metrics.update(output.metrics)
        return PredictionOutput(predictions=predictions.predictions, label_ids=predictions.label_ids, metrics=metrics)

In [8]:
max_length = 1000
global global_doc_stride
#global_doc_stride = 128
pretrained_model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'

## The dataset does not have the ending index of the "Answers" which is required for the Question and Answering transformer.
## This function uses the data in the Squad dataset to extract the start index and end index as well as tokenizes the dataset.
## after running this function, the data is ready to be used in the model

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second" if right_padding else "only_first",
        stride=global_doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        if len(answer["answer_start"]) > 0:
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
        else:
            start_char = -1
            end_char = -1
        sequence_ids = inputs.sequence_ids(i)
        
        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx -1
        

        # No answer
        if start_char == -1 and end_char == -1:
            start_positions.append(0)
            end_positions.append(0)
        # If the answer is not fully inside the context, label is (0, 0)
        elif offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 2)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs



In [9]:
def prepare_validation_features(examples):
  # Some of the questions have lots of whitespace on the left, which is not useful and will make the
  # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
  # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]
  # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
  # in one example possible giving several features when a context is long, each of those features having a
  # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
      examples["question"],
      examples["context"],
      truncation="only_second" if right_padding else "only_first",
      max_length=max_length,
      stride=global_doc_stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
  )

  # Since one example might give us several features if it has a long context, we need a map from a feature to
  # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

  # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
  # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
      # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 0

      # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

      # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
      # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
      ]

    return tokenized_examples

In [10]:
logger = logging.getLogger(__name__)

def postprocess_qa_predictions(
    examples,
    features,
    predictions: Tuple[np.ndarray, np.ndarray],
    version_2_with_negative: bool = True,
    n_best_size: int = 20,
    max_answer_length: int = 30,
    null_score_diff_threshold: float = 0.0,
    output_dir: Optional[str] = None,
    prefix: Optional[str] = None,
    log_level: Optional[int] = logging.WARNING,
):
    """
    Post-processes the predictions of a question-answering model to convert them to answers that are substrings of the
    original contexts. This is the base postprocessing functions for models that only return start and end logits.

    Args:
        examples: The non-preprocessed dataset (see the main script for more information).
        features: The processed dataset (see the main script for more information).
        predictions (:obj:`Tuple[np.ndarray, np.ndarray]`):
            The predictions of the model: two arrays containing the start logits and the end logits respectively. Its
            first dimension must match the number of elements of :obj:`features`.
        version_2_with_negative (:obj:`bool`, `optional`, defaults to :obj:`False`):
            Whether or not the underlying dataset contains examples with no answers.
        n_best_size (:obj:`int`, `optional`, defaults to 20):
            The total number of n-best predictions to generate when looking for an answer.
        max_answer_length (:obj:`int`, `optional`, defaults to 30):
            The maximum length of an answer that can be generated. This is needed because the start and end predictions
            are not conditioned on one another.
        null_score_diff_threshold (:obj:`float`, `optional`, defaults to 0):
            The threshold used to select the null answer: if the best answer has a score that is less than the score of
            the null answer minus this threshold, the null answer is selected for this example (note that the score of
            the null answer for an example giving several features is the minimum of the scores for the null answer on
            each feature: all features must be aligned on the fact they `want` to predict a null answer).

            Only useful when :obj:`version_2_with_negative` is :obj:`True`.
        output_dir (:obj:`str`, `optional`):
            If provided, the dictionaries of predictions, n_best predictions (with their scores and logits) and, if
            :obj:`version_2_with_negative=True`, the dictionary of the scores differences between best and null
            answers, are saved in `output_dir`.
        prefix (:obj:`str`, `optional`):
            If provided, the dictionaries mentioned above are saved with `prefix` added to their names.
        log_level (:obj:`int`, `optional`, defaults to ``logging.WARNING``):
            ``logging`` log level (e.g., ``logging.WARNING``)
    """
    if len(predictions) != 2:
        raise ValueError("`predictions` should be a tuple with two elements (start_logits, end_logits).")
    all_start_logits, all_end_logits = predictions

    if len(predictions[0]) != len(features):
        raise ValueError(f"Got {len(predictions[0])} predictions and {len(features)} features.")

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    all_predictions = collections.OrderedDict()
    all_nbest_json = collections.OrderedDict()
    if version_2_with_negative:
        scores_diff_json = collections.OrderedDict()

    # Logging.
    logger.setLevel(log_level)
    logger.info(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_prediction = None
        prelim_predictions = []

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]
            # Optional `token_is_max_context`, if provided we will remove answers that do not have the maximum context
            # available in the current feature.
            token_is_max_context = features[feature_index].get("token_is_max_context", None)

            # Update minimum null prediction.
            feature_null_score = start_logits[0] + end_logits[0]
            if min_null_prediction is None or min_null_prediction["score"] > feature_null_score:
                min_null_prediction = {
                    "offsets": (0, 0),
                    "score": feature_null_score,
                    "start_logit": start_logits[0],
                    "end_logit": end_logits[0],
                }

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or len(offset_mapping[start_index]) < 2
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[end_index]) < 2
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    # Don't consider answer that don't have the maximum context available (if such information is
                    # provided).
                    if token_is_max_context is not None and not token_is_max_context.get(str(start_index), False):
                        continue

                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0], offset_mapping[end_index][1]),
                            "score": start_logits[start_index] + end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )
        if version_2_with_negative and min_null_prediction is not None:
            # Add the minimum null prediction
            prelim_predictions.append(min_null_prediction)
            null_score = min_null_prediction["score"]

        # Only keep the best `n_best_size` predictions.
        predictions = sorted(prelim_predictions, key=lambda x: x["score"], reverse=True)[:n_best_size]

        # Add back the minimum null prediction if it was removed because of its low score.
        if (
            version_2_with_negative
            and min_null_prediction is not None
            and not any(p["offsets"] == (0, 0) for p in predictions)
        ):
            predictions.append(min_null_prediction)

        # Use the offsets to gather the answer text in the original context.
        context = example["context"]
        for pred in predictions:
            offsets = pred.pop("offsets")
            pred["text"] = context[offsets[0] : offsets[1]]

        # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
        # failure.
        if len(predictions) == 0 or (len(predictions) == 1 and predictions[0]["text"] == ""):
            predictions.insert(0, {"text": "empty", "start_logit": 0.0, "end_logit": 0.0, "score": 0.0})

        # Compute the softmax of all scores (we do it with numpy to stay independent from torch/tf in this file, using
        # the LogSumExp trick).
        scores = np.array([pred.pop("score") for pred in predictions])
        exp_scores = np.exp(scores - np.max(scores))
        probs = exp_scores / exp_scores.sum()

        # Include the probabilities in our predictions.
        for prob, pred in zip(probs, predictions):
            pred["probability"] = prob

        # Pick the best prediction. If the null answer is not possible, this is easy.
        if not version_2_with_negative:
            all_predictions[example["id"]] = predictions[0]["text"]
        else:
            # Otherwise we first need to find the best non-empty prediction.
            i = 0
            while predictions[i]["text"] == "":
                i += 1
            best_non_null_pred = predictions[i]

            # Then we compare to the null prediction using the threshold.
            score_diff = null_score - best_non_null_pred["start_logit"] - best_non_null_pred["end_logit"]
            scores_diff_json[example["id"]] = float(score_diff)  # To be JSON-serializable.
            if score_diff > null_score_diff_threshold:
                all_predictions[example["id"]] = ""
            else:
                all_predictions[example["id"]] = best_non_null_pred["text"]

        # Make `predictions` JSON-serializable by casting np.float back to float.
        all_nbest_json[example["id"]] = [
            {k: (float(v) if isinstance(v, (np.float16, np.float32, np.float64)) else v) for k, v in pred.items()}
            for pred in predictions
        ]

    # If we have an output_dir, let's save all those dicts.
    if output_dir is not None:
        if not os.path.isdir(output_dir):
            raise EnvironmentError(f"{output_dir} is not a directory.")

        prediction_file = os.path.join(
            output_dir, "predictions.json" if prefix is None else f"{prefix}_predictions.json"
        )
        nbest_file = os.path.join(
            output_dir, "nbest_predictions.json" if prefix is None else f"{prefix}_nbest_predictions.json"
        )
        if version_2_with_negative:
            null_odds_file = os.path.join(
                output_dir, "null_odds.json" if prefix is None else f"{prefix}_null_odds.json"
            )

        logger.info(f"Saving predictions to {prediction_file}.")
        with open(prediction_file, "w") as writer:
            writer.write(json.dumps(all_predictions, indent=4) + "\n")
        logger.info(f"Saving nbest_preds to {nbest_file}.")
        with open(nbest_file, "w") as writer:
            writer.write(json.dumps(all_nbest_json, indent=4) + "\n")
        if version_2_with_negative:
            logger.info(f"Saving null_odds to {null_odds_file}.")
            with open(null_odds_file, "w") as writer:
                writer.write(json.dumps(scores_diff_json, indent=4) + "\n")

    return all_predictions

In [11]:
def post_processing_function(examples, features, predictions, stage="eval"):
        # Post-processing: we match the start logits and end logits to answers in the original context.
        predictions = postprocess_qa_predictions(
            examples=examples,
            features=features,
            predictions=predictions,
            max_answer_length=max_length
        )
        # Format the result to the format the metric expects.
        if 1==1:
            formatted_predictions = [
                {"id": str(k), "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
            ]
        else:
            formatted_predictions = [{"id": str(k), "prediction_text": v} for k, v in predictions.items()]

        references = [{"id": str(ex["id"]), "answers": ex["answers"]} for ex in examples]
        return EvalPrediction(predictions=formatted_predictions, label_ids=references)

In [12]:
class AdvancedEarlyStoppingCallback(TrainerCallback):
    """
    A callback to stop training when either the performance falls below a certain threshold
    or if there is no improvement over a set number of epochs.
    """
    def __init__(self, metric_name, patience, threshold):
        self.metric_name = metric_name
        self.patience = patience
        self.threshold = threshold
        self.best_score = None
        self.no_improve_epochs = 0

    def on_evaluate(self, args, state, control, **kwargs):
        metric_value = kwargs['metrics'].get(self.metric_name)

        if self.best_score is None or metric_value > self.best_score:
            self.best_score = metric_value
            self.no_improve_epochs = 0
        else:
            self.no_improve_epochs += 1

        # Check if performance is below the threshold
        if metric_value < self.threshold:
            control.should_training_stop = True
            print(f"Stopping training: {self.metric_name} below threshold of {self.threshold}")

        # Check if no improvement has been seen over the allowed patience
        if self.no_improve_epochs >= self.patience:
            control.should_training_stop = True
            print(f"Stopping training: No improvement in {self.metric_name} for {self.patience} epochs")


In [13]:
# Define model initialization function
def model_init():
    return AutoModelForQuestionAnswering.from_pretrained(pretrained_model_name)

# Define train dataset initialization function
def train_dataset_init():
    return squad_raw['train'].map(
                preprocess_function,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on train dataset",
            )

# Define validation dataset initialization function
def vald_dataset_init():
    return squad_raw['validation'].map(
                prepare_validation_features,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on validation dataset",
            )

# Optuna objective function for hyperparameter tuning
def objective(trial):
    # Hyperparameters to tune    
    learning_rate = trial.suggest_float('learning_rate', 1e-7, 1e-4, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    warmup_steps = trial.suggest_int('warmup_steps', 0, 1000)
    weight_decay = trial.suggest_float('weight_decay', 0.01, 0.25)
    adam_beta1 = trial.suggest_float('adam_beta1', 0.8, 0.95)
    adam_beta2 = trial.suggest_float('adam_beta2', 0.990, 0.999)
    adam_epsilon = trial.suggest_float('adam_epsilon', 1e-8, 1e-6)
    lr_scheduler_type = trial.suggest_categorical('lr_scheduler_type', ['linear', 'cosine', 'cosine_with_restarts','constant_with_warmup'])
    output_dir = f"./{normalized_model_name}-finetuned-squadv2/trial_{trial.number}"
    
    #global global_doc_stride
    #global_doc_stride=trial.suggest_int('doc_stride', 128, 256, step=64)

    # Print trial parameters
    print(f"Current Trial {trial.number} parameters: {trial.params}")
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir = True,
        metric_for_best_model='f1',
        greater_is_better=True,
        load_best_model_at_end=True,
        save_total_limit=2, # Save only the best model unless you specify a different number
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=10,  # Adjust based on computation limits
        report_to="wandb",  # Enable logging to Weights & Biases        
        run_name=f"{normalized_model_name}-finetune-squadv2",  # Optionally set a specific run name    
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        lr_scheduler_type=lr_scheduler_type,
        fp16=True,  # Enable mixed-precision training
    )    

    trainer = QuestionAnsweringTrainer(
        model=model_init(),
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=train_dataset_init(),
        eval_dataset=vald_dataset_init(),
        eval_examples=eval_examples,        
        data_collator=data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,
        #callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
        callbacks=[AdvancedEarlyStoppingCallback(metric_name='eval_f1', patience=1, threshold=45)]
    )  
    

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    #print("Evaluation results:", eval_results)  # Debug print
    return eval_results['eval_f1']


## 1. Data Preparation

In [131]:
max_length = 512
global global_doc_stride
#global_doc_stride = 128
pretrained_model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'

## The dataset does not have the ending index of the "Answers" which is required for the Question and Answering transformer.
## This function uses the data in the Squad dataset to extract the start index and end index as well as tokenizes the dataset.
## after running this function, the data is ready to be used in the model

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation=True,
        stride=global_doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        if len(answer["answer_start"]) > 0:
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
        else:
            start_char = -1
            end_char = -1
        sequence_ids = inputs.sequence_ids(i)
        
        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx -1
        

        # No answer
        if start_char == -1 and end_char == -1:
            start_positions.append(0)
            end_positions.append(0)
        # If the answer is not fully inside the context, label is (0, 0)
        elif offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 2)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs



In [ ]:
max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

# Training preprocessing
    def prepare_train_features(examples):
        # Some of the questions have lots of whitespace on the left, which is not useful and will make the
        # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
        # left whitespace
        examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]

        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=data_args.doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length" if data_args.pad_to_max_length else False,
        )

        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        offset_mapping = tokenized_examples.pop("offset_mapping")

        # Let's label those examples!
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []

        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            if tokenizer.cls_token_id in input_ids:
                cls_index = input_ids.index(tokenizer.cls_token_id)
            elif tokenizer.bos_token_id in input_ids:
                cls_index = input_ids.index(tokenizer.bos_token_id)
            else:
                cls_index = 0

            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples[answer_column_name][sample_index]
            # If no answers are given, set the cls_index as answer.
            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                    token_start_index += 1

                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                    token_end_index -= 1

                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)

        return tokenized_examples

In [266]:
max_length = 512  # Adjust max_length to fit model constraints
global global_doc_stride
#global_doc_stride = 128
pretrained_model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance(tokenizer, PreTrainedTokenizerFast)
pad_on_right = tokenizer.padding_side == 'right'

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,  # Updated to handle model max length
        truncation="only_second" if pad_on_right else "only_first",
        stride=global_doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = tokenized_examples.pop("offset_mapping")
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        if tokenizer.cls_token_id in input_ids:
            cls_index = input_ids.index(tokenizer.cls_token_id)
        elif tokenizer.bos_token_id in input_ids:
            cls_index = input_ids.index(tokenizer.bos_token_id)
        else:
            cls_index = 0
        
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples['answers'][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)                

    return tokenized_examples
        

In [236]:
## IMPORT RAW DATASET

squad_dataset = load_dataset("squad_v2", split="train[:2000]")
train_testvalid = squad_dataset.train_test_split(test_size=0.2)

test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

squad_raw = datasets.DatasetDict({
                                'train': train_testvalid['train'],
                                'validation': test_valid['train'],
                                'test': test_valid['test']
                                })

In [121]:
squad_raw

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [260]:
global_doc_stride = 128
global train_dataset
global eval_dataset
## APPLY PREPROCESSING
train_dataset = squad_raw['train'].map(
                preprocess_function,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on train dataset",
            )
eval_dataset = squad_raw['validation'].map(
                prepare_validation_features,
                batched=True,
                remove_columns=squad_raw["train"].column_names,
                desc="Running tokenizer on validation dataset",
            )
eval_examples =  squad_raw["validation"]


Running tokenizer on train dataset:   0%|          | 0/1600 [00:00<?, ? examples/s]

In [262]:
def retrieve_and_compare_answers(dataset, examples, split_type='train'):
    answer_mismatches = []
    counter = 0
    for i, example in enumerate(examples):
        # Retrieve stored start and end positions
        start_pos = dataset[i]['start_positions']
        end_pos = dataset[i]['end_positions']
        
        # Fetch the context and calculate predicted answer text
        context = example['context']
        question = example['question'] 
        answer_text = tokenizer.decode(dataset[i]['input_ids'][start_pos:end_pos])
        
        # Normalize and compare with actual answer
        actual_answer = example['answers']['text'][0] if example['answers']['text'] else ""
        #normalized_actual_answer = unidecode(actual_answer.lower().replace(" ", ""))
        #normalized_predicted_answer = unidecode(answer_text.lower().replace(" ", ""))
        normalized_actual_answer = actual_answer.lower().replace(" ", "")
        normalized_predicted_answer = answer_text.lower().replace(" ", "")
        
        if normalized_actual_answer != normalized_predicted_answer:
            counter += 1
            answer_mismatches.append({
                'example_id': example['id'],
                'decoded_answer': answer_text,
                'actual_answer': actual_answer,
                'context': context,
                'question': question
            })
        #if counter > 300: break
    
    print(f"Number of mismatches in {split_type}: {len(answer_mismatches)}")
    return answer_mismatches

# Retrieve mismatches for training and validation datasets
train_mismatches = retrieve_and_compare_answers(train_dataset, squad_raw['train'], 'train')

# Optionally, review some mismatches
print(train_mismatches[:12])  # Print first 3 mismatches from training
#print(validation_mismatches[:3])  # Print first 3 mismatches from validation


Number of mismatches in train: 1562
[{'example_id': '56d39bdf59d6e41400146808', 'decoded_answer': '', 'actual_answer': '1833', 'context': 'The 21 nocturnes are more structured, and of greater emotional depth, than those of Field (whom Chopin met in 1833). Many of the Chopin nocturnes have middle sections marked by agitated expression (and often making very difficult demands on the performer) which heightens their dramatic character.', 'question': 'What year did Chopin meet Field?'}, {'example_id': '56cc84ab6d243a140015efe3', 'decoded_answer': '1833', 'actual_answer': 'individual seat-back displays', 'context': 'Beginning in mid-2007, four major airlines, United, Continental, Delta, and Emirates, reached agreements to install iPod seat connections. The free service will allow passengers to power and charge an iPod, and view video and music libraries on individual seat-back displays. Originally KLM and Air France were reported to be part of the deal with Apple, but they later released st

In [264]:
def retrieve_and_compare_answers(dataset, examples, split_type='train'):
    answer_mismatches = []
    counter = 0
    for i, example in enumerate(examples):
        # Retrieve stored start and end positions
        start_pos = dataset[i]['start_positions']
        end_pos = dataset[i]['end_positions']
        
        # Fetch the context and calculate predicted answer text
        context = example['context']
        question = example['question']
        answer_text = tokenizer.decode(dataset[i]['input_ids'][start_pos:end_pos])
        
        # Normalize and compare with actual answer
        actual_answer = example['answers']['text'][0] if example['answers']['text'] else ""
        normalized_actual_answer = unidecode(actual_answer.lower().replace(" ", ""))
        normalized_predicted_answer = unidecode(answer_text.lower().replace(" ", ""))
        
        if normalized_actual_answer != normalized_predicted_answer:
            counter += 1
            answer_mismatches.append({
                'Raw Example ID': example['id'],
                'Raw Question': question,
                'Decoded Question': question,
                'Decoded Answer': answer_text,
                'Actual Answer': actual_answer,
                'Raw Context': context[:200] + '...'  # Truncating context for display purposes
            })
        if counter > 50:
            break
    
    # Convert list of dictionaries to DataFrame
    mismatches_df = pd.DataFrame(answer_mismatches)
    print(f"Number of mismatches in {split_type}: {len(answer_mismatches)}")
    return mismatches_df

# Retrieve mismatches for training and validation datasets
train_mismatches = retrieve_and_compare_answers(train_dataset, squad_raw['train'], 'train')
# Uncomment and adjust as needed for validation dataset
#validation_mismatches = retrieve_and_compare_answers(eval_dataset, squad_raw['validation'], 'validation')

# Optionally, display some mismatches in a DataFrame
display(train_mismatches.tail(50))  # Display first 12 mismatches from training
#display(validation_mismatches.head(3))  # Display first 3 mismatches from validation if needed


Number of mismatches in train: 51


,Raw Example ID,Raw Question,Decoded Question,Decoded Answer,Actual Answer,Raw Context
1,56cc84ab6d243a140015efe3,Where can people using iPods on planes view th...,Where can people using iPods on planes view th...,1833,individual seat-back displays,"Beginning in mid-2007, four major airlines, Un..."
2,56d43da72ccc5a1400d830c1,What is the name of Beyoncé's alter-ego?,What is the name of Beyoncé's alter-ego?,individual seat - back displays,Sasha Fierce,Following the disbandment of Destiny's Child i...
3,56cd687562d2951400fa6592,When did HP unveil their own edition of the iPod?,When did HP unveil their own edition of the iPod?,Sasha Fierce,"January 8, 2004","On January 8, 2004, Hewlett-Packard (HP) annou..."
4,56be9add3aeaaa14008c9156,"in 2011, Beyonce performed for four nights where?","in 2011, Beyonce performed for four nights where?","January 8, 2004",New York's Roseland Ballroom,Her fourth studio album 4 was released on June...
5,56cf582eaab44d1400b8909c,Chopin composed several songs to lyrics of wha...,Chopin composed several songs to lyrics of wha...,New York's Roseland Ballroom,Polish,All of Chopin's compositions include the piano...
6,56d45fcb2ccc5a1400d830fb,What was Destiny's Child's first major song hit?,What was Destiny's Child's first major song hit?,Polish,"No, No, No",The group changed their name to Destiny's Chil...
7,56bf725c3aeaaa14008c9644,What magazine rated Beyonce as the most powerf...,What magazine rated Beyonce as the most powerf...,"No, No, No",Forbes,"A self-described ""modern-day feminist"", Beyonc..."
8,56cf67c74df3c31400b0d72e,Which singer did Chopin become fascinated with?,Which singer did Chopin become fascinated with?,Forbes,Konstancja Gładkowska,Four boarders at his parents' apartments becam...
9,56d3a85259d6e414001468ac,What music did Debussy play a lot at the Paris...,What music did Debussy play a lot at the Paris...,Konstancja Gładkowska,Chopin's,"Two of Chopin's long-standing pupils, Karol Mi..."
10,56d38c2b59d6e41400146705,Which cemetery was Chopin buried in?,Which cemetery was Chopin buried in?,Chopin's,Père Lachaise Cemetery,Mozart's Requiem was sung at the funeral; the ...


In [57]:
df = pd.DataFrame(squad_raw['train'])
id = '56cdcf7d62d2951400fa686e'
print(df[df.id==id].question.values)
print(df[df.id==id].context.values)
print(df[df.id==id].answers.values)

["Who is M's rival?"]
['Following Garreth Mallory\'s promotion to M, on a mission in Mexico City unofficially ordered by a posthumous message from the previous M, 007 James Bond kills three men plotting a terrorist bombing during the Day of the Dead and gives chase to Marco Sciarra, an assassin who survived the attack. In the ensuing struggle, Bond steals his ring, which is emblazoned with a stylised octopus, and then kills Sciarra by kicking him out of a helicopter. Upon returning to London, Bond is indefinitely suspended from field duty by M, who is in the midst of a power struggle with C, the head of the privately-backed Joint Intelligence Service, consisting of the recently merged MI5 and MI6. C campaigns for Britain to form alongside 8 other countries "Nine Eyes ", a global surveillance and intelligence co-operation initiative between nine member states, and uses his influence to close down the \'00\' section, believing it to be outdated.']
[{'text': ['C'], 'answer_start': [67]}]


In [151]:
def retrieve_and_compare_answers(dataset, examples, split_type='train'):
    answer_mismatches = []
    counter = 0
    for i, example in enumerate(examples):
        # Retrieve stored start and end positions
        start_pos = dataset[i]['start_positions']
        end_pos = dataset[i]['end_positions']
        
        # Fetch the context and calculate predicted answer text
        context = example['context']
        question = example['question']
        answer_text = tokenizer.decode(dataset[i]['input_ids'][start_pos:end_pos])
        
        # Normalize and compare with actual answer
        actual_answer = example['answers']['text'][0] if example['answers']['text'] else ""
        normalized_actual_answer = unidecode(actual_answer.lower().replace(" ", ""))
        normalized_predicted_answer = unidecode(answer_text.lower().replace(" ", ""))
        
        # Tokenize to count the number of tokens
        token_count_question = len(tokenizer.tokenize(question))
        token_count_answer = len(tokenizer.tokenize(answer_text))
        token_count_context = len(tokenizer.tokenize(context))
        
        if normalized_actual_answer != normalized_predicted_answer:
            answer_mismatches.append({
                'Example ID': example['id'],
                'Total Tokens count' : token_count_question + token_count_answer + token_count_context,
                'Question': question,
                'Decode Answer': answer_text,
                'Actual Answer': actual_answer,
                'start_pos':start_pos,
                'end_pos':end_pos,
                'Context': context[:200] + '...'  # Truncating context for display purposes
            })
            counter += 1
            if counter > 50: break
    
    # Convert list of dictionaries to DataFrame
    mismatches_df = pd.DataFrame(answer_mismatches)
    print(f"Number of mismatches in {split_type}: {len(answer_mismatches)}")
    return mismatches_df

# Retrieve mismatches for training and validation datasets
train_mismatches = retrieve_and_compare_answers(train_dataset, squad_raw['train'], 'train')
# Uncomment and adjust as needed for validation dataset
#validation_mismatches = retrieve_and_compare_answers(eval_dataset, squad_raw['validation'], 'validation')

# Optionally, display some mismatches in a DataFrame
display(train_mismatches.head(20))  # Display first 12 mismatches from training
#display(validation_mismatches.head(3))  # Display first 3 mismatches from validation if needed


Number of mismatches in train: 51


,Example ID,Total Tokens count,Question,Decode Answer,Actual Answer,start_pos,end_pos,Context
0,56be85543aeaaa14008c9063,173,When did Beyonce start becoming popular?,in the late,in the late 1990s,71,74,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,56be85543aeaaa14008c9065,177,What areas did Beyonce compete in when she was...,singing and,singing and dancing,64,66,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,56be85543aeaaa14008c9066,177,When did Beyonce leave Destiny's Child and bec...,,2003,142,142,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,56bf6b0f3aeaaa14008c9601,175,In what city and state did Beyonce grow up?,"Houston,","Houston, Texas",54,56,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4,56bf6b0f3aeaaa14008c9602,172,In which decade did Beyonce become famous?,late,late 1990s,74,75,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
5,56bf6b0f3aeaaa14008c9603,176,In what R&B group was she the lead singer?,Destiny's,Destiny's Child,88,91,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
6,56bf6b0f3aeaaa14008c9604,173,What album made her a worldwide known artist?,Dangerously in,Dangerously in Love,130,133,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
7,56bf6b0f3aeaaa14008c9605,172,Who managed the Destiny's Child group?,Mathew,Mathew Knowles,96,98,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
8,56d43c5f2ccc5a1400d830a9,169,When did Beyoncé rise to fame?,late,late 1990s,71,72,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
9,56d43c5f2ccc5a1400d830aa,173,What role did Beyoncé have in Destiny's Child?,lead,lead singer,78,79,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...


## 2. Train BERT Model


In [227]:
###############################################################
# UNCOMMENT THIS TO TEST TRAINER BEFORE STARTING TUNING
###############################################################
pretrained_model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'

data_collator = DefaultDataCollator()
model = AutoModelForQuestionAnswering.from_pretrained(pretrained_model_name)
training_args = TrainingArguments(
    output_dir=f'{pretrained_model_name}-finetuned-manual',
    overwrite_output_dir = True,
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
    save_total_limit=4, 
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="wandb",  # Enable logging to Weights & Biases
    run_name=f"{pretrained_model_name}-finetune-manual",  # Optionally set a specific run name    
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

trainer = QuestionAnsweringTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        eval_examples=eval_examples,
        tokenizer=tokenizer,
        data_collator=data_collator,
        post_process_function=post_processing_function,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [19]:
# Clear cache if using GPU
if torch.cuda.is_available():
    torch.cuda.empty_cache()
import gc

# Delete unnecessary variables
del trainer
del model
del tokenizer
del training_args
del data_collator

# Collect garbage
gc.collect()
from tensorflow.keras.backend import clear_session

# Clear TensorFlow session
clear_session()


## Tune BERT Model


In [21]:
# Tokenizer  initialisation
max_length = 870
#global_doc_stride = 128
pretrained_model_name = "bert-base-uncased"
normalized_model_name = pretrained_model_name.replace("/", "-")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'
data_collator = DefaultDataCollator()


In [22]:
# Create a study object and optimize the objective
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2024-05-25 04:07:29,827] A new study created in memory with name: no-name-db7b923a-638c-4277-9a20-68de3385eea1


Current Trial 0 parameters: {'learning_rate': 5.272153980495028e-07, 'batch_size': 32, 'warmup_steps': 859, 'weight_decay': 0.04620569945443442, 'adam_beta1': 0.8256891086545727, 'adam_beta2': 0.9909432216963988, 'adam_epsilon': 1.4016712084494723e-07, 'lr_scheduler_type': 'cosine'}


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Noans Exact,Noans F1,Noans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,2.482500,No log,0.505348,0.521765,11873,0.000000,0.032881,5928,1.009251,1.009251,5945,50.071591,0.000000,50.071591,0.000000


  0%|          | 0/11873 [00:00<?, ?it/s]

...

Current Trial 16 parameters: {'learning_rate': 3.023120268076598e-07, 'batch_size': 64, 'warmup_steps': 493, 'weight_decay': 0.05651478873758259, 'adam_beta1': 0.8914514746506399, 'adam_beta2': 0.9972158518028947, 'adam_epsilon': 4.696620940030543e-07, 'lr_scheduler_type': 'constant_with_warmup'}


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


[W 2024-05-25 10:50:16,045] Trial 16 failed with parameters: {'learning_rate': 3.023120268076598e-07, 'batch_size': 64, 'warmup_steps': 493, 'weight_decay': 0.05651478873758259, 'adam_beta1': 0.8914514746506399, 'adam_beta2': 0.9972158518028947, 'adam_epsilon': 4.696620940030543e-07, 'lr_scheduler_type': 'constant_with_warmup'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\OEM\AppData\Local\Temp\ipykernel_1644\3488163911.py", line 83, in objective
    trainer.train()
  File "C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\trainer.py", line 1885, in train
    return inner_training_loop(
  File "C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\trainer.py", line 2221, in _inner_training_loop
    and (torch.isnan(tr_loss_step) or to

KeyboardInterrupt: 

In [ ]:
# Clear cache if using GPU
if torch.cuda.is_available():
    torch.cuda.empty_cache()
import gc

# Delete unnecessary variables
del data_collator
del tokenizer
del study

# Collect garbage
gc.collect()
from tensorflow.keras.backend import clear_session

# Clear TensorFlow session
clear_session()


## 4. Tune RoBERTa Model


In [ ]:
# Tokenizer  initialisation
max_length = 870
#global_doc_stride = 128
pretrained_model_name = "bert-base-uncased"
normalized_model_name = pretrained_model_name.replace("/", "-")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'
data_collator = DefaultDataCollator()


In [ ]:
# Create a study object and optimize the objective
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

In [ ]:
torch.cuda.empty_cache()

## 5. Tune XLNet Model


In [ ]:
# Tokenizer  initialisation
max_length = 870
#global_doc_stride = 128
pretrained_model_name = "xlnet/xlnet-base-cased"
normalized_model_name = pretrained_model_name.replace("/", "-")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'
data_collator = DefaultDataCollator()


In [ ]:
# Create a study object and optimize the objective
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

In [ ]:
torch.cuda.empty_cache()

## 3. Tune Facebook RoBERTa Model


In [ ]:

# Tokenizer  initialisation
max_length = 870
#global_doc_stride = 128
pretrained_model_name = "FacebookAI/roberta-base"
normalized_model_name = pretrained_model_name.replace("/", "-")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'
data_collator = DefaultDataCollator()


In [ ]:
# Create a study object and optimize the objective
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

In [ ]:
torch.cuda.empty_cache()

## 6. Tune AlBERT Model

In [ ]:

# Tokenizer  initialisation
max_length = 870
#global_doc_stride = 128
pretrained_model_name = "albert/albert-base-v2"
normalized_model_name = pretrained_model_name.replace("/", "-")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
right_padding = tokenizer.padding_side == 'right'
data_collator = DefaultDataCollator()


In [ ]:
# Create a study object and optimize the objective
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

In [ ]:
torch.cuda.empty_cache()